# Brain Tumor MRI Classification – End-to-End Demo

Runs the repo **brain-tumor-hybrid-fusion-knn** with the tiny embedded demo (10 images per class).

**What this notebook does**  
1. Clone / update repo  
2. Install deps  
3. Verify dataset  
4. Write `configs/config.yaml`  
5. **Fix `src` imports** (adds `__init__.py` + path injection)  
6. Run split-check & full pipeline  
7. Display results  

---

### 1. Idempotent repo clone / update

In [ ]:
%%bash
set -e
cd /content

REPO="brain-tumor-hybrid-fusion-knn"
if [ -d "$REPO/.git" ]; then
  echo "Pulling latest main…"
  cd "$REPO"
  git fetch origin
  git reset --hard origin/main
  git clean -fd
else
  echo "Cloning fresh repo…"
  git clone https://github.com/mainajajere/brain-tumor-hybrid-fusion-knn.git "$REPO"
  cd "$REPO"
fi

echo "Repo ready at $(pwd)"

### 2. Install dependencies + GPU check

In [ ]:
!pip install -q tensorflow==2.17.0 scikit-learn==1.4.2 matplotlib==3.8.4 seaborn==0.13.2
!pip install -q opencv-python-headless==4.9.0.80 Pillow==10.3.0 numpy==1.26.4 pandas==2.1.4
!pip install -q pyyaml==6.0.1 tqdm==4.66.4 shap==0.46.0

import os, sys, subprocess, pathlib

REPO = pathlib.Path('/content/brain-tumor-hybrid-fusion-knn')
os.chdir(REPO)
sys.path.insert(0, str(REPO))

os.makedirs(REPO/'outputs', exist_ok=True)
os.makedirs(REPO/'results', exist_ok=True)

print('Repo ready at', REPO)

result = subprocess.run(["bash", "-lc", "nvidia-smi -L"], capture_output=True, text=True)
if result.returncode == 0:
    print(result.stdout.strip())
    print('GPU available')
else:
    print('No GPU (CPU mode)')

### 3. Verify embedded demo dataset

In [ ]:
DATA_ROOT = '/content/brain-tumor-hybrid-fusion-knn/data/images'
CLASSES = ['glioma','meningioma','pituitary','notumor']

print('Dataset root:', DATA_ROOT)
for c in CLASSES:
    p = os.path.join(DATA_ROOT, c)
    n = len(os.listdir(p)) if os.path.isdir(p) else 0
    print(f'{c}: {n} images' if n else f'{c}: MISSING')

if all(os.path.isdir(os.path.join(DATA_ROOT,c)) and len(os.listdir(os.path.join(DATA_ROOT,c))) for c in CLASSES):
    print('Dataset OK')
else:
    raise SystemExit('Demo dataset missing')

### 4. Write config

In [ ]:
import yaml

cfg = {
    'data': {'root_dir': DATA_ROOT, 'classes': CLASSES, 'image_size': [224,224], 'seed': 42,
             'split': {'test': 0.20, 'val_from_train': 0.20}},
    'augment': {'rotation': 0.055, 'zoom': 0.10, 'translate': 0.10, 'hflip': True, 'contrast': 0.15},
    'train': {'batch_size': 32, 'epochs': 50, 'optimizer': 'adam', 'lr': 0.001, 'dropout': 0.5},
    'fusion': {'type': 'late', 'pooling': 'gap', 'concat': True},
    'knn': {'n_neighbors': 5, 'metric': 'euclidean', 'weights': 'distance'},
    'cv': {'n_folds': 5, 'stratify': True},
    'xai': {'shap_background_per_class': 25}
}

os.makedirs('configs', exist_ok=True)
with open('configs/config.yaml','w') as f:
    yaml.safe_dump(cfg, f, sort_keys=False)
print('Config written: configs/config.yaml')

### 5. **Fix `src` imports** (the magic that kills the `ModuleNotFoundError`)

In [ ]:
# 5.1 – Go to repo root & add to Python path (for the notebook)
import os, sys
os.chdir('/content/brain-tumor-hybrid-fusion-knn')
sys.path.insert(0, os.getcwd())
print('Repo root:', os.getcwd())
print('src visible?', os.path.isdir('src'))

# 5.2 – Verify dataset.py exists
print('Expect src/data/dataset.py →', os.path.exists('src/data/dataset.py'))

# 5.3 – Make src a real package (safe if already there)
os.makedirs('src/data', exist_ok=True)
open('src/__init__.py', 'a').close()
open('src/data/__init__.py', 'a').close()
print('src package initialized')

# 5.4 – Inject path-fix at the top of both scripts (so they work anywhere)
inject = "import os, sys\nROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))\nif ROOT not in sys.path: sys.path.insert(0, ROOT)\n"

for script in ['scripts/check_split_counts.py', 'scripts/run_full_pipeline.py']:
    with open(script, 'r', encoding='utf-8') as f:
        txt = f.read()
    if not txt.startswith('import os, sys'):
        with open(script, 'w', encoding='utf-8') as f:
            f.write(inject + txt)
        print('Patched', script)
    else:
        print('Already patched', script)

# 5.5 – Quick sanity-check (first 10 lines of each script)
!sed -n '1,10p' scripts/check_split_counts.py
!sed -n '1,10p' scripts/run_full_pipeline.py

### 6. Run split-check & full pipeline

In [ ]:
print("=== Split check ===")
!PYTHONPATH=/content/brain-tumor-hybrid-fusion-knn python -m scripts.check_split_counts --config configs/config.yaml

print("\n=== Full pipeline ===")
!PYTHONPATH=/content/brain-tumor-hybrid-fusion-knn python -m scripts.run_full_pipeline --config configs/config.yaml

### 7. Show results

In [ ]:
from IPython.display import Image, display
import os

print("=== Pipeline Outputs ===")
for p in ['outputs/figures/confusion_matrix.png',
          'outputs/figures/class_metrics.png',
          'outputs/results/summary.txt']:
    print('\n' + p)
    if p.endswith('.png') and os.path.exists(p):
        display(Image(filename=p))
    elif os.path.exists(p):
        print(open(p).read())
    else:
        print('Not generated')

if os.path.exists('outputs'):
    print('\nAll output files:')
    !find outputs -type f | head -15